In [ ]:
!pip install torchsde
from google.colab import userdata  # Import userdata to access secrets
from huggingface_hub import login
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
from IPython.display import Audio  # Import the Audio module for playback

# Retrieve the Hugging Face token from Colab Secrets
try:
    huggingface_token = userdata.get('HF_TOKEN')
except Exception as e:
    raise ValueError("Please ensure the 'my_token' secret is set in Colab Secrets.") from e

# Log in to Hugging Face using the token
login(token=huggingface_token)

# Load the StableAudioPipeline
pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Define the prompts
prompt = "Elephant noises mix with wildlife noise"
negative_prompt = "mechanical, artificial, synthetic, human voice, music, rhythm, Low quality."

# Set the seed for the generator
generator = torch.Generator("cuda")

# Run the generation
audio = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=200,
    audio_end_in_s=12.0,
    num_waveforms_per_prompt=3,
    generator=generator,
).audios

# Save the generated audio to a file
output = audio[0].T.float().cpu().numpy()
sf.write("output.wav", output, pipe.vae.sampling_rate)

# Play the audio in Colab
Audio("output.wav")